In [ ]:
%matplotlib inline  
%reload_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

from access_literature_data import medline
from access_science_shared import standardizer

import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('./../src/')
import resci_tools as ret

In [ ]:
save_images = True

In [ ]:
sns.set(font_scale=2)

In [ ]:
taxon_id = 9606

ref_genes = standardizer.reference_genes(taxon_id, 'orp')  # offical nomenclature, research, protein-coding
gene2pubmed = medline.gene2pubmed(taxon_id, ['pubmed_id', 'gene_ncbi'], paper_kind='research', ref_genes=ref_genes)

In [ ]:
ml = medline.select_medline_records(
        columns_sql='''
            medline.pubmed_id,
            medline.pubdate_year''',
        taxon_id=taxon_id,
        kind='research')

In [ ]:
gene2pubmed = pd.merge(gene2pubmed, ml)

In [ ]:
h = gene2pubmed['pubmed_id'].value_counts()
single_gene = list(h[h==1].index)              ######## ATTENTION
r = gene2pubmed.loc[:, 'pubmed_id'].isin(single_gene)
gene2pubmed.loc[:, 'is_single'] = r

In [ ]:
gene2pubmed = gene2pubmed[gene2pubmed['pubdate_year'] <= 2015]

In [ ]:
fract_single =gene2pubmed[['pubmed_id', 'pubdate_year', 'is_single']].drop_duplicates(
    ).drop('pubmed_id', 1)

In [ ]:
# ax = sns.pointplot(x='pubdate_year',y='is_single', data=fract_single)

# if save_images:
#     ret.export_image('180111_papers_per_year/fraction_single_gene_human.pdf')

In [ ]:
an = gene2pubmed.loc[gene2pubmed['is_single']==True, ['gene_ncbi', 'pubdate_year']]

In [ ]:
span =np.arange(1980, 2101)

In [ ]:
mins = [1, 2, 5, 10]

In [ ]:
res = pd.DataFrame(index=span, columns=mins)

In [ ]:
for m in mins:
    for y in sorted(span):
        f = an['pubdate_year']==y
        g = an.loc[f, 'gene_ncbi']
        res.loc[y, m] = np.count_nonzero(g.value_counts()>=m)

In [ ]:
res=res / len(ref_genes) * 100

In [ ]:
ax = res.loc[span].plot()
ax.set_ylim((0,100))

ax.set_xlabel('Year')
ax.set_ylabel('Genes with n paper(s) [%]')

if save_images:
    ret.export_image('180111_papers_per_year/n_paper_human.pdf')